In [19]:
!pip install bcrypt

In [26]:
from tkinter import *
import tkinter as tk
from tkinter import messagebox
import sqlite3
import bcrypt

# Hàm băm mật khẩu
def hash_password(password):
    password_bytes = password.encode('utf-8')
    salt = bcrypt.gensalt()
    hashed_password = bcrypt.hashpw(password_bytes, salt)
    return hashed_password

# Hàm xác minh mật khẩu
def verify_password(stored_password, provided_password):
    provided_password_bytes = provided_password.encode('utf-8')
    return bcrypt.checkpw(provided_password_bytes, stored_password)

    

# Tạo cơ sở dữ liệu và bảng người dùng
def tao_database():
    try:
        conn = sqlite3.connect('bankaccount.db')
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS users (
                username VARCHAR(255) PRIMARY KEY,
                password VARCHAR(255) NOT NULL,
                balance INTEGER
            )
        ''')
        conn.commit()
        print("Cơ sở dữ liệu và bảng người dùng đã được tạo thành công.")
    except sqlite3.Error as e:
        print(f"Lỗi khi tạo cơ sở dữ liệu: {e}")
    finally:
        if conn:
            conn.close()

# Lớp BankAccount dùng để lưu trữ tài khoản
class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.balance = balance

    def display_format(self):
        return '{:,}'.format(int(self.balance)).replace(',', '.')

    def amount_format(self, amount):
        return '{:,}'.format(int(amount)).replace(',', '.')
        
#Hàm gửi tiền vào tài khoản
    def deposit(self, amount):
        self.balance += amount
        self.update_balance_db()
        messagebox.showinfo('Thành công', f'Đã nạp {self.amount_format(amount)} VND vào tài khoản.\nSố dư hiện tại: {self.display_format()} VND')

#Hàm rút tiền khỏi tài khoản
    def withdraw(self, amount):
        if self.balance >= amount:
            if self.balance - amount >= 50000:
                self.balance -= amount
                self.update_balance_db()
                messagebox.showinfo('Thành công!', f'Đã rút {self.amount_format(amount)} VND từ tài khoản.\nSố dư hiện tại: {self.display_format()} VND')
            else:
                messagebox.showerror('Lỗi', 'Số dư tài khoản phải nhiều hơn 50.000 VND')
        else:
            messagebox.showerror('Lỗi', f'Số tiền trong tài khoản {self.owner} không đủ để giao dịch')

#Hàm xem thông tin tài khoản
    def display_balance(self):
        messagebox.showinfo('Thông tin tài khoản', f'Chủ tài khoản: {self.owner}\nSố dư tài khoản: {self.display_format()} VND')

    def update_balance_db(self):
        conn = sqlite3.connect('bankaccount.db')
        cursor = conn.cursor()
        cursor.execute('UPDATE users SET balance = ? WHERE username = ?', (self.balance, self.owner))
        conn.commit()
        conn.close()

    @classmethod
    def signup(cls, username_entry, password_entry, confirm_password_entry, signup_window):
        username = username_entry.get()
        password = password_entry.get()
        conpw = confirm_password_entry.get()
        if not username or not password or not conpw:
            messagebox.showerror('Lỗi', 'Vui lòng điền đầy đủ thông tin')
            return
        conn = sqlite3.connect('bankaccount.db')
        cursor = conn.cursor()
        cursor.execute('SELECT username FROM users WHERE username = ?', (username,))
        result = cursor.fetchone()
        conn.close()
        if result:
            messagebox.showerror('Lỗi', 'Tên đăng nhập đã tồn tại')
            return
        if password != conpw:
            messagebox.showerror('Lỗi', 'Mật khẩu không trùng khớp')
            return
        hashed_password = hash_password(password)
        conn = sqlite3.connect('bankaccount.db')
        cursor = conn.cursor()
        cursor.execute('INSERT INTO users (username, password, balance) VALUES (?, ?, ?)', (username, hashed_password, 0))
        conn.commit()
        conn.close()
        messagebox.showinfo('Thành công', 'Đã tạo tài khoản thành công!')
        signup_window.destroy()
        loginScreen()

    @classmethod
    def login(cls, username_entry, password_entry, login_window):
        username = username_entry.get()
        password = password_entry.get()
        if not username or not password:
            messagebox.showerror('Lỗi', 'Vui lòng điền đầy đủ tên đăng nhập và mật khẩu')
            return
        conn = sqlite3.connect('bankaccount.db')
        cursor = conn.cursor()
        cursor.execute('SELECT password, balance FROM users WHERE username = ?', (username,))
        result = cursor.fetchone()
        conn.close()
        if result and verify_password(result[0], password):
            messagebox.showinfo('Thành công', 'Đăng nhập thành công!')
            login_window.destroy()
            mainmenu(username, result[1])
        else:
            messagebox.showerror('Lỗi', 'Tên đăng nhập hoặc mật khẩu không đúng')

#Màn hình chính sau khi đăng nhập thành công
def mainmenu(username, balance):
    MainScreen = screen()
    title(MainScreen)
    MainMenu = Label(MainScreen, text='Menu chính:', font=('Arial', 20, 'bold'), fg='white', bg='#c644a1')
    MainMenu.place(x=20, y=130)

    account = BankAccount(username, balance)

    def display_balance_command():
        account.display_balance()

    def deposit_command():
        deposit_window(account)

    def withdraw_command():
        withdraw_window(account)

    ThongTinKH = tk.Button(MainScreen, text='Thông tin khách hàng', font=('Arial', 10), width=20, command=display_balance_command)
    ThongTinKH.place(x=50, y=170)
    GuiTien = tk.Button(MainScreen, text='Nạp tiền vào tài khoản', font=('Arial', 10), width=20, command=deposit_command)
    GuiTien.place(x=50, y=210)
    RutTien = tk.Button(MainScreen, text='Rút tiền từ tài khoản', font=('Arial', 10), width=20, command=withdraw_command)
    RutTien.place(x=50, y=250)
    DangXuat = tk.Button(MainScreen, text='Đăng xuất', font=('Arial', 10), padx=5, command=lambda: logout(MainScreen))
    DangXuat.place(x=210, y=310)
    LoginSuc = Label(MainScreen, text=f'Chào mừng {username}!', font=('Arial', 12, 'bold'), fg='white', bg='#aa2b61')
    MainScreen.update_idletasks()
    screen_width = MainScreen.winfo_width()
    label_width = LoginSuc.winfo_reqwidth()
    x_position = (screen_width - label_width) // 2
    LoginSuc.place(x=x_position, y=350)
    MainScreen.mainloop()

if __name__ == "__main__":
    tao_database()
    loginScreen()

Cơ sở dữ liệu và bảng người dùng đã được tạo thành công.
